<a href="https://colab.research.google.com/github/LaboraDev/hackathon_one_flightOnTime/blob/main/DOC_DS3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Relatório de Feature Engineering - Semana 03 (DS3)

**Detalhes do Projeto**
*   **Responsáveis:** Amélia e Ana, Time DS (Liderança: Helena)
*   **Repositório:** [Projeto FlightOnTime](https://github.com/LaboraDev/hackathon_one_flightOnTime/tree/main)
*   **Conformidade:** Boas práticas de Data Science, Prevenção de *Data Leakage* e Metodologia [D15](https://www.cirium.com/resources/on-time-performance/on-time-performance-faq/).
*   **Notebook Rastreável:** [Consolidado_S03_splitTemporal.ipynb](https://github.com/LaboraDev/hackathon_one_flightOnTime/blob/main/notebooks/semana03/Consolidado_S03_splitTemporal.ipynb)
*   **Eficiência Técnica:** Pipeline modularizado e reprodutível via script [`flightdelaypipeline.py`](https://github.com/LaboraDev/hackathon_one_flightOnTime/blob/main/notebooks/semana_05/flight_delay_pipeline.py)
*   **Status:** Finalizado | **Data:** 16/01/2026

***

## 1. Visão Geral

Durante a Semana 03, construímos o [pipeline de transformação](https://github.com/LaboraDev/hackathon_one_flightOnTime/blob/main/notebooks/semana03/Consolidado_S03_splitTemporal.ipynb) que converte dados brutos em variáveis preditivas (*features*). [in [5]](https://github.com/LaboraDev/hackathon_one_flightOnTime/blob/main/notebooks/semana03/Consolidado_S03_splitTemporal.ipynb) O objetivo foi criar um sistema estruturado que captura padrões de sazonalidade e gargalos operacionais da malha aérea brasileira, usando técnicas de engenharia de tempo cíclico e codificação inteligente de alta cardinalidade. Este trabalho garante que o modelo aprenda com o passado sem "vazar" informações do futuro, preparando a base para predições confiáveis de atraso de voos no projeto [FlightOnTime](https://github.com/LaboraDev/hackathon_one_flightOnTime/tree/main).

***

## 2. Dicionário de Terminologia Técnica
Para garantir a compreensão de todos, usamos as seguintes definições:
- **Feature Engineering:** Processo de criar novas variáveis a partir dos dados brutos para melhorar o aprendizado do modelo. [in [5]](https://github.com/LaboraDev/hackathon_one_flightOnTime/blob/main/notebooks/semana03/Consolidado_S03_splitTemporal.ipynb)
- **Data Leakage:** Vazamento de informação do futuro para o passado que torna o modelo inutilizável na prática.
- **Split Temporal:** Divisão dos dados respeitando a ordem cronológica (treino com dados antigos, teste com dados recentes). [in [8]](https://github.com/LaboraDev/hackathon_one_flightOnTime/blob/main/notebooks/semana03/Consolidado_S03_splitTemporal.ipynb)
- **Target Encoding:** Técnica que substitui categorias por médias calculadas da variável alvo. [in [6] ](https://github.com/LaboraDev/hackathon_one_flightOnTime/blob/main/notebooks/semana03/Consolidado_S03_splitTemporal.ipynb)
- **One-Hot Encoding:** Transformação de categorias em colunas binárias (0 ou 1). **
- **Cardinalidade:** Quantidade de valores únicos de uma variável (ex: aeroportos têm alta cardinalidade com +600 valores).

***

## 3. Grupos de Features Criadas
Para capturar diferentes aspectos da operação aérea, criamos três famílias de variáveis com propósitos específicos. [in [5] ](https://github.com/LaboraDev/hackathon_one_flightOnTime/blob/main/notebooks/semana03/Consolidado_S03_splitTemporal.ipynb) :

| Grupo | Tipo | Propósito | Exemplos |
| :--- | :--- | :--- | :--- |
| **Temporais Cíclicas** | Numéricas | Capturar repetição do tempo (hora 23 próxima de hora 0). | `hora_sin`, `hora_cos`, `dia_semana` |
| **Logística e Hubs** | Categóricas | Identificar infraestruturas críticas e períodos de alta demanda. | `is_hub`, `periodo_dia`, `alta_temporada` |
| **Estatísticas Agregadas** | Numéricas | Representar histórico de desempenho de rotas/empresas. | `media_atraso_empresa`, `media_atraso_origem` |

***

## 4. Funil de Transformação e Score de Integridade

### 4.1 Pipeline de Processamento
| Etapa da Transformação | Técnica Aplicada | Dimensão Final | Justificativa Técnica |
| :--- | :--- | :--- | :--- |
| **Entrada de Dados Estruturados** | Dataset da Semana 01 | 17 colunas | Base saneada e validada. |
| **Split Temporal** | Divisão cronológica | 3 conjuntos | Treino (2021-2024), Val (2024), Teste (2024-2025). |
| **Extração Temporal** | Criação de `hora`, `dia_semana`, `mes` | +3 colunas | Padrões de atraso variam por horário. |
| **Codificação Cíclica** | Seno/Cosseno para hora | +2 colunas | Continuidade temporal (23h próxima de 0h). |
| **Target Encoding** | Médias de atraso por entidade | +2 colunas | Reduz dimensionalidade de centenas de categorias. |
| **One-Hot Encoding** | Expansão de categóricas | +531 colunas | Tratamento de baixa cardinalidade. |
| **Dataset Final Processado** | **555 colunas** | - | Base pronta para treinamento de modelos. |

### 4.2 Score de Prevenção de Data Leakage
| Dimensão | Score | Observação Técnica |
| :--- | :--- | :--- |
| **Split Temporal** | 10/10 | Dados divididos cronologicamente, modelo nunca vê o futuro. |
| **Cálculo de Médias** | 10/10 | Estatísticas agregadas calculadas APENAS no treino. |
| **Exclusão de Variáveis** | 10/10 | Colunas de "chegada" e "real" removidas do conjunto X. |
| **Validação Cruzada** | 10/10 | TimeSeriesSplit garante ordem temporal nos folds. |

***

## 5. Regras e decisões do projeto
Estabelecemos os seguintes pontos fundamentais para garantir a integridade do pipeline de transformação:

*   **Ordem Temporal Rigorosa:** Todo o processamento respeita a cronologia. O modelo só aprende com dados de períodos anteriores ao que está prevendo. [in [10]](https://github.com/LaboraDev/hackathon_one_flightOnTime/blob/main/notebooks/semana03/Consolidado_S03_splitTemporal.ipynb)
*   **Médias Históricas Isoladas:** As features de `media_atraso` são calculadas sobre o conjunto de treino e replicadas para validação/teste, impedindo contaminação de informação futura. [in [9]](https://github.com/LaboraDev/hackathon_one_flightOnTime/blob/main/notebooks/semana03/Consolidado_S03_splitTemporal.ipynb)
*   **Codificação Cíclica do Tempo:** Utilizamos funções trigonométricas (seno e cosseno) para representar a hora do dia, garantindo que 23h seja matematicamente próxima de 0h.
*   **Descarte de Variáveis Futuras:** Qualquer coluna que contenha informação posterior ao momento da partida prevista (como `partidareal`, `chegadareal`) é removida para evitar *leakage* total.
*   **Pipeline Reprodutível:** Todas as transformações são encapsuladas em classes Python reutilizáveis no módulo [`flightdelaypipeline.py`](https://github.com/LaboraDev/hackathon_one_flightOnTime/blob/main/notebooks/semana_05/flight_delay_pipeline.py)

***

## 6. Respostas às Perguntas-Chave

### 6.1 Quais colunas entram no pipeline como dados brutos?
As variáveis de entrada são as 17 colunas do dataset estruturado da Semana 01: `empresaaerea`, `aerodromoorigem`, `aerodromodestino`, `partidaprevista`, `situacaovoo`, `codigotipolinha`, entre outras. Estas servem de matéria-prima para todas as transformações subsequentes. [in [10]](https://github.com/LaboraDev/hackathon_one_flightOnTime/blob/main/notebooks/semana03/Consolidado_S03_splitTemporal.ipynb)

### 6.2 Quais transformações temporais são aplicadas?
Extraímos componentes temporais como `hora`, `dia_semana` e `mes` da coluna `partidaprevista`. Em seguida, aplicamos funções trigonométricas seno e cosseno na hora para criar `hora_sin` e `hora_cos`, permitindo que o modelo capture a ciclicidade do tempo (ex: 23h é próximo de 0h). [in [11]](https://github.com/LaboraDev/hackathon_one_flightOnTime/blob/main/notebooks/semana03/Consolidado_S03_splitTemporal.ipynb)

### 6.3 Como variáveis categóricas são tratadas?
Aplicamos duas técnicas complementares:
- **One-Hot Encoding:** Para variáveis com poucos valores únicos (ex: `situacaovoo` com 3 categorias). [in ](https://github.com/LaboraDev/hackathon_one_flightOnTime/blob/main/notebooks/semana03/Consolidado_S03_splitTemporal.ipynb)
- **Target Encoding:** Para variáveis com centenas de valores (ex: `aerodromoorigem` com +600 aeroportos), substituímos cada categoria pela média histórica de atraso daquela entidade, evitando a explosão dimensional. [in [6]](https://github.com/LaboraDev/hackathon_one_flightOnTime/blob/main/notebooks/semana03/Consolidado_S03_splitTemporal.ipynb)

### 6.4 Existem features agregadas? Com base em quê?
Sim. Criamos `media_atraso_empresa` e `media_atraso_origem`, que representam o desempenho histórico de cada companhia aérea e aeroporto de origem. Essas médias são calculadas **exclusivamente** sobre os dados de treino para evitar vazamento de informação do conjunto de teste. **[in [12] ](https://github.com/LaboraDev/hackathon_one_flightOnTime/blob/main/notebooks/semana03/Consolidado_S03_splitTemporal-1.ipynb)

### 6.5 Como o pipeline evita vazamento de informação (*Data Leakage*)?
O vazamento é evitado através de três mecanismos rigorosos:
1. **Split Temporal:** Dados divididos cronologicamente (treino até abril/2024, teste de setembro/2024 em diante). [in ](https://github.com/LaboraDev/hackathon_one_flightOnTime/blob/main/notebooks/semana03/Consolidado_S03_splitTemporal-1.ipynb)
2. **Cálculo Isolado de Estatísticas:** Médias e agregações extraídas apenas do conjunto de treino. [in ](https://github.com/LaboraDev/hackathon_one_flightOnTime/blob/main/notebooks/semana03/Consolidado_S03_splitTemporal-1.ipynb)
3. **Remoção de Features Futuras:** Colunas como `partidareal` e `chegadareal` são descartadas antes do treinamento.

### 6.6 Alguma feature foi descartada? Por quê?
Sim. Foram descartadas:
- **`codigojustificativa`:** Campo 100% nulo desde abril/2020.
- **Variáveis de "Realização":** `partidareal`, `chegadareal` contêm informações que só existem **após** o evento que queremos prever, causando *leakage* total se incluídas no conjunto X. [in [4]](https://github.com/LaboraDev/hackathon_one_flightOnTime/blob/main/notebooks/semana03/Consolidado_S03_splitTemporal.ipynb)


Esta abordagem preserva a integridade temporal e evita que o modelo "trapaceie" usando informações do futuro.

### 6.7 Quais limitações ainda existem nas features?
- **Ausência de Clima:** O dataset não possui dados meteorológicos em tempo real, um grande fator de atraso não capturado.
- **Mudanças de Malha:** Comportamentos históricos de 2021 podem não se repetir em 2025 devido a mudanças na infraestrutura ou rotas.
- **Aeroportos Novos:** Aeroportos sem histórico suficiente terão médias zeradas ou próximas de zero, reduzindo a precisão para essas localidades. [in [14]](https://github.com/LaboraDev/hackathon_one_flightOnTime/blob/main/notebooks/semana03/Consolidado_S03_splitTemporal.ipynb)


***

## 7. Análise de Riscos para Modelagem

| Risco | Severidade | Mitigation Strategy |
| :--- | :--- | :--- |
| **Ausência de Clima** | Média | Focar em padrões temporais e geográficos históricos robustos. |
| **Alta Cardinalidade** | Baixa | Target Encoding aplicado para reduzir dimensionalidade sem perda. |
| **Aeroportos Novos** | Baixa | Imputação com média global ou valores conservadores. |
| **Mudanças de Malha** | Média | Split temporal atenua, mas não elimina completamente o risco. |

***


## 8. Checklist de Integridade (Selo de Qualidade)
- [x] **Prevenção de Leakage:** Split temporal rigoroso e cálculo de médias isolado no treino.
- [x] **Reprodutibilidade:** Pipeline encapsulado em classes Python reutilizáveis.
- [x] **Escalabilidade:** Técnicas de codificação otimizadas para alta cardinalidade (+600 aeroportos).
- [x] **Rastreabilidade:** Mapeamento direto entre narrativa e células de código do notebook.
- [x] **Documentação:** Todas as transformações explicadas em linguagem acessível.

***

## 9. Blueprint de Rastreabilidade
Para auditoria técnica e validação, utilize as referências abaixo no notebook [`Consolidado_S03_splitTemporal.ipynb`](https://github.com/LaboraDev/hackathon_one_flightOnTime/blob/main/notebooks/semana03/Consolidado_S03_splitTemporal.ipynb):

| Etapa | Células | Descrição |
| :--- | :--- | :--- |
| **Importação do Dataset** | [1] - [3] |  Carregamento da base estruturada da Semana 01. |
| **Split Temporal** |  [5] | Divisão cronológica dos dados (treino, validação, teste). |
| **Importação do Pipeline** |  [6] | Carregamento do módulo [`flightdelaypipeline.py`](https://github.com/LaboraDev/hackathon_one_flightOnTime/blob/main/notebooks/semana_05/flight_delay_pipeline.py) |
| **Definição de Features** | [8] | Configuração de listas de variáveis numéricas e categóricas. |
| **Extração Temporal** | [9]  | Criação de `hora`, `dia_semana`, `mes` e codificação cíclica. |
| **Target Encoding** | [10] | Cálculo de médias de atraso por empresa e origem. |
| **One-Hot Encoding** | [11] | Expansão de variáveis categóricas de baixa cardinalidade. |
| **Validação Final** | [12] - [14] | Verificação de dimensões (555 colunas) e ausência de leakage. |

***

## 10. Conclusão

### Validação da Semana 03
O pipeline de Feature Engineering foi consolidado com sucesso. O dataset transformado possui:
- **555 colunas processadas** com técnicas de engenharia temporal e geográfica.
- **Nenhum vazamento de dados** (conformidade total com boas práticas de ML).
- **Pipeline reprodutível** encapsulado em funções reutilizáveis no script Python.
- **Redução de dimensionalidade** através de Target Encoding para alta cardinalidade.


***

## Referências
- ANAC - Agência Nacional de Aviação Civil. (2024). Portal de Dados Abertos - VRA. Disponível em: [ANAC](https://www.gov.br/anac/pt-br/acesso-a-informacao/dados-abertos/areas-de-atuacao/voos-e-operacoes-aereas/voo-regular-ativo-vra)
- Scikit-learn. (2024). Feature Engineering and Preprocessing. Disponível em: [Scikit-learn](https://scikit-learn.org/)
- IATA - International Air Transport Association. (2023). D15 Punctuality Standard. Disponível em: [D15 (padrão IATA)](https://www.cirium.com/resources/on-time-performance/on-time-performance-faq/)
- Lei Geral de Proteção de Dados (LGPD). Lei nº 13.709, de 14 de agosto de 2018. Brasil. Disponível em: [LGPD](https://www.planalto.gov.br/ccivil_03/_ato2015-2018/2018/lei/l13709.htm)


